In [2]:
import sys
import pandas as pd
import numpy as np
import os
import io
import matplotlib.pyplot as plt
import matplotlib.colors
import json
from urllib.request import urlopen
import dash
import plotly_express as px
import plotly.graph_objects as go
import nbformat
import json
import urllib
import seaborn as sns
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.neighbors import KNeighborsClassifier as KNNc
import sklearn.metrics as Metrics
import re 
import nltk
from shapely.geometry import Point
import cufflinks as cf
cf.go_offline()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.probability import FreqDist
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import pickle
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode

In [4]:
rev= pd.read_csv("/Users/fondairenasantoso/airbnb_data_analysis/data/reviews1.csv")
ny = pd.read_csv("/Users/fondairenasantoso/airbnb_data_analysis/cleaned_airbnb_listings.csv")

In [5]:
merged_rev = ny[['id','property_type','host_id']].merge(rev[['id','listing_id','comments','reviewer_id']],on='id',how='left')

In [6]:
merged_rev = merged_rev.dropna(subset=['comments'], how='any')

In [7]:
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [8]:
def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)
merged_rev['tokenized_column'] = merged_rev.comments.apply(custom_tokenize)

In [9]:
merged_rev

,id,property_type,host_id,listing_id,comments,reviewer_id,tokenized_column
59,17037.0,Apartment,66035.0,5803.0,A huge thank you to Laurie for 'being there' w...,42873.0,"[A, huge, thank, you, to, Laurie, for, 'being,..."
85,23545.0,Townhouse,92411.0,8110.0,Our family of 5 (three nine year old children ...,63099.0,"[Our, family, of, 5, (, three, nine, year, old..."
163,45936.0,Bed and breakfast,867225.0,21293.0,Silvia made made my trip perfect. The appartme...,121488.0,"[Silvia, made, made, my, trip, perfect, ., The..."
306,84010.0,Boutique hotel,417504.0,9782.0,"good place, nice price, everything OK!",185403.0,"[good, place, ,, nice, price, ,, everything, O..."
384,141335.0,House,687361.0,29628.0,We have been very satisfied by the ten days sp...,228033.0,"[We, have, been, very, satisfied, by, the, ten..."
...,...,...,...,...,...,...,...
42678,40134329.0,Apartment,229332707.0,1284776.0,It was just great to stay at this apartment in...,21481630.0,"[It, was, just, great, to, stay, at, this, apa..."
43573,40623330.0,Apartment,314533486.0,1953816.0,This apartment was absolutely beautiful and we...,34171341.0,"[This, apartment, was, absolutely, beautiful, ..."
48216,43194706.0,Apartment,282955633.0,1095472.0,Lee was a kind and welcoming host. Her descrip...,18211453.0,"[Lee, was, a, kind, and, welcoming, host, ., H..."
48859,43427720.0,Apartment,167332602.0,4951151.0,Ben's place was great. Heart of soho near ever...,22729959.0,"[Ben, 's, place, was, great, ., Heart, of, soh..."


In [12]:
merged_rev['tokenized_column'] = merged_rev['tokenized_column'].str.lower()

In [14]:
merged_rev['comments'] = merged_rev['comments'].str.lower()

In [20]:
def identify_tokens(row):
    comments = row['comments']
    tokens = nltk.word_tokenize(comments)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

In [18]:
merged_rev

,id,property_type,host_id,listing_id,comments,reviewer_id,tokenized_column
59,17037.0,Apartment,66035.0,5803.0,a huge thank you to laurie for 'being there' w...,42873.0,NaN
85,23545.0,Townhouse,92411.0,8110.0,our family of 5 (three nine year old children ...,63099.0,NaN
163,45936.0,Bed and breakfast,867225.0,21293.0,silvia made made my trip perfect. the appartme...,121488.0,NaN
306,84010.0,Boutique hotel,417504.0,9782.0,"good place, nice price, everything ok!",185403.0,NaN
384,141335.0,House,687361.0,29628.0,we have been very satisfied by the ten days sp...,228033.0,NaN
...,...,...,...,...,...,...,...
42678,40134329.0,Apartment,229332707.0,1284776.0,it was just great to stay at this apartment in...,21481630.0,NaN
43573,40623330.0,Apartment,314533486.0,1953816.0,this apartment was absolutely beautiful and we...,34171341.0,NaN
48216,43194706.0,Apartment,282955633.0,1095472.0,lee was a kind and welcoming host. her descrip...,18211453.0,NaN
48859,43427720.0,Apartment,167332602.0,4951151.0,ben's place was great. heart of soho near ever...,22729959.0,NaN


In [21]:
merged_rev['comments'] = merged_rev.apply(identify_tokens, axis=1)

In [22]:
merged_rev

,id,property_type,host_id,listing_id,comments,reviewer_id,tokenized_column
59,17037.0,Apartment,66035.0,5803.0,"[a, huge, thank, you, to, laurie, for, there, ...",42873.0,NaN
85,23545.0,Townhouse,92411.0,8110.0,"[our, family, of, three, nine, year, old, chil...",63099.0,NaN
163,45936.0,Bed and breakfast,867225.0,21293.0,"[silvia, made, made, my, trip, perfect, the, a...",121488.0,NaN
306,84010.0,Boutique hotel,417504.0,9782.0,"[good, place, nice, price, everything, ok]",185403.0,NaN
384,141335.0,House,687361.0,29628.0,"[we, have, been, very, satisfied, by, the, ten...",228033.0,NaN
...,...,...,...,...,...,...,...
42678,40134329.0,Apartment,229332707.0,1284776.0,"[it, was, just, great, to, stay, at, this, apa...",21481630.0,NaN
43573,40623330.0,Apartment,314533486.0,1953816.0,"[this, apartment, was, absolutely, beautiful, ...",34171341.0,NaN
48216,43194706.0,Apartment,282955633.0,1095472.0,"[lee, was, a, kind, and, welcoming, host, her,...",18211453.0,NaN
48859,43427720.0,Apartment,167332602.0,4951151.0,"[ben, place, was, great, heart, of, soho, near...",22729959.0,NaN


In [24]:
#stemming to test accuracy 
stemming = PorterStemmer()

def stem_list(row):
    my_list = row['comments']
    stemmed_list = [stemming.stem(comment) for comment in my_list]
    return (stemmed_list)

merged_rev['stemmed_comments'] = merged_rev.apply(stem_list, axis=1)

In [25]:
merged_rev

,id,property_type,host_id,listing_id,comments,reviewer_id,tokenized_column,stemmed_comments
59,17037.0,Apartment,66035.0,5803.0,"[a, huge, thank, you, to, laurie, for, there, ...",42873.0,NaN,"[a, huge, thank, you, to, lauri, for, there, w..."
85,23545.0,Townhouse,92411.0,8110.0,"[our, family, of, three, nine, year, old, chil...",63099.0,NaN,"[our, famili, of, three, nine, year, old, chil..."
163,45936.0,Bed and breakfast,867225.0,21293.0,"[silvia, made, made, my, trip, perfect, the, a...",121488.0,NaN,"[silvia, made, made, my, trip, perfect, the, a..."
306,84010.0,Boutique hotel,417504.0,9782.0,"[good, place, nice, price, everything, ok]",185403.0,NaN,"[good, place, nice, price, everyth, ok]"
384,141335.0,House,687361.0,29628.0,"[we, have, been, very, satisfied, by, the, ten...",228033.0,NaN,"[we, have, been, veri, satisfi, by, the, ten, ..."
...,...,...,...,...,...,...,...,...
42678,40134329.0,Apartment,229332707.0,1284776.0,"[it, was, just, great, to, stay, at, this, apa...",21481630.0,NaN,"[it, wa, just, great, to, stay, at, thi, apart..."
43573,40623330.0,Apartment,314533486.0,1953816.0,"[this, apartment, was, absolutely, beautiful, ...",34171341.0,NaN,"[thi, apart, wa, absolut, beauti, and, we, lov..."
48216,43194706.0,Apartment,282955633.0,1095472.0,"[lee, was, a, kind, and, welcoming, host, her,...",18211453.0,NaN,"[lee, wa, a, kind, and, welcom, host, her, des..."
48859,43427720.0,Apartment,167332602.0,4951151.0,"[ben, place, was, great, heart, of, soho, near...",22729959.0,NaN,"[ben, place, wa, great, heart, of, soho, near,..."


In [26]:
#remove stopwords

def remove_stops(row):
    my_list = row['stemmed_comments']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

merged_rev['stem_comments'] = merged_rev.apply(remove_stops, axis=1)

In [29]:
merged_rev.head()

,id,property_type,host_id,listing_id,comments,reviewer_id,tokenized_column,stemmed_comments,stem_comments
59,17037.0,Apartment,66035.0,5803.0,"[a, huge, thank, you, to, laurie, for, there, ...",42873.0,NaN,"[a, huge, thank, you, to, lauri, for, there, w...","[huge, thank, lauri, much, advic, info, local,..."
85,23545.0,Townhouse,92411.0,8110.0,"[our, family, of, three, nine, year, old, chil...",63099.0,NaN,"[our, famili, of, three, nine, year, old, chil...","[famili, three, nine, year, old, children, plu..."
163,45936.0,Bed and breakfast,867225.0,21293.0,"[silvia, made, made, my, trip, perfect, the, a...",121488.0,NaN,"[silvia, made, made, my, trip, perfect, the, a...","[silvia, made, made, trip, perfect, appart, wa..."
306,84010.0,Boutique hotel,417504.0,9782.0,"[good, place, nice, price, everything, ok]",185403.0,NaN,"[good, place, nice, price, everyth, ok]","[good, place, nice, price, everyth, ok]"
384,141335.0,House,687361.0,29628.0,"[we, have, been, very, satisfied, by, the, ten...",228033.0,NaN,"[we, have, been, veri, satisfi, by, the, ten, ...","[veri, satisfi, ten, day, spent, apart, christ..."


In [30]:
merged_rev.to_pickle('cleaned_reviews.pkl')